In [65]:
# import the libraries we will use
import numpy as np               # for array and matrix maths
import pandas as pd              # for working with tables of data
import matplotlib.pyplot as plt  # for making plots

In [66]:
# Load the Kaggle german credit data: https://www.kaggle.com/uciml/german-credit/data
df_kaggle = pd.read_csv("~/data/german/german_credit_data.csv", index_col=0)
# Load the original UCI german credit data: https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29
df_uci = pd.read_csv("~/data/german/german.data", header=None, sep="\s+")

# Add the last column from the UCI data to the kaggle data to get labels
last_column_uci = df_uci.iloc[:, -1]
df = df_kaggle.assign(Good=last_column_uci)

In [67]:
category_cols = ['Sex', 'Job', 'Housing', 'Purpose']
target_col = 'Good'

for c in category_cols + [target_col]:
    df[c] = df[c].astype('category')

def uci_class_as_boolean(x):
    if x == 1:
        return True
    elif x == 2:
        return False
    else:
        return x
    
df[target_col] = df[target_col].apply(uci_class_as_boolean)
df = df[pd.notna(df['Saving accounts'])]

In [68]:
df['Saving accounts'].unique()

array(['little', 'quite rich', 'rich', 'moderate'], dtype=object)

In [69]:
account_value_lookup = {
    np.nan: 0,
    'little': 1,
    'moderate': 2,
    'rich': 3,
    'quite rich': 4
}

def dollar_dollar(x):
    return account_value_lookup[x]

df['Saving amount'] = df['Saving accounts'].apply(dollar_dollar)

In [70]:
import pystan

In [49]:
model_code = """
data {
    int<lower=0> N;

    int<lower=0, upper=1> y[N];

    int female[N];
    vector[N] bank_balance;
    
    real<lower=0> money_mean_prior_std;
    real<lower=0> money_per_u_prior_std;

    real<lower=0> good_alpha_prior_std;
    real<lower=0> good_beta_prior_std;
    
    real<lower=0> bank_balance_std;
}

parameters {
    real money_mean_male;
    real money_mean_female;
    real money_per_u;

    real good_alpha;
    real good_beta;

    vector[N] u;
}

model {
    money_mean_female ~ normal(0, money_mean_prior_std);
    money_mean_male ~ normal(0, money_mean_prior_std);
    money_per_u ~ normal(0, money_per_u_prior_std);

    good_alpha ~ normal(0, good_alpha_prior_std);
    good_beta ~ normal(0, good_beta_prior_std);

    u ~ normal(0, 1);

    for (n in 1:N) {
        real pred_balance;
        if (female[n]) {
            pred_balance = money_mean_female + money_per_u*u[n];
        } else {
            pred_balance = money_mean_male + money_per_u*u[n];
        }
        bank_balance[n] ~ normal(pred_balance, bank_balance_std);
    }
    
    y ~ bernoulli_logit(bank_balance * good_beta + good_alpha + u);
}
"""

In [50]:
model = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_dd88a3a980ba9f56451dd3bed951ee71 NOW.


In [71]:
data = dict(
    N = len(df),
    y = df['Good'].values.astype(np.int),
    female = (df['Sex'] == 'female').values.astype(np.int),
    bank_balance = df['Saving amount'].values,
    money_mean_prior_std = 4.0,
    money_per_u_prior_std = 4.0,
    good_alpha_prior_std = 1.0,
    good_beta_prior_std = 1.0,
    bank_balance_std = 1.0)

In [72]:
fit = model.sampling(data)

In [73]:
fit


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_dd88a3a980ba9f56451dd3bed951ee71.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                    mean se_mean     sd   2.5%    25%    50%     75%  97.5%  n_eff   Rhat
money_mean_male     1.48  5.6e-4   0.04    1.4   1.46   1.48    1.51   1.57   5815    1.0
money_mean_female   1.45  8.3e-4   0.06   1.33   1.41   1.45    1.49   1.58   5680    1.0
money_per_u       2.8e-4  3.0e-3   0.08  -0.16  -0.05 4.9e-4    0.05   0.16    729   1.01
good_alpha          0.24  6.4e-3    0.2  -0.16    0.1   0.24    0.38   0.63   1011    1.0
good_beta           0.44  4.3e-3   0.13   0.19   0.35   0.44    0.53   0.71    946    1.0
u[1]               -0.53    0.01    0.9   -2.3  -1.12  -0.53    0.07   1.26   5278    1.0
u[2]                0.29    0.01    0.9   -1.4  -0.33   0.28     0.9   2.08   6670    1.0
u[3]                 0.3    0.01   0.92  -1.45  -0.